In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ===================================
# 1. Logistic Map Data
# ===================================
def logistic_map(r, x0, n_steps):
    x = np.zeros(n_steps)
    x[0] = x0
    for i in range(1, n_steps):
        x[i] = r * x[i-1] * (1 - x[i-1])
    return x

np.random.seed(42)
r = 4.0
x0 = 0.1
total_steps = 10000
data = logistic_map(r, x0, total_steps)

train_data = data[:6000]
test_data = data[6000:8000]

# ===================================
# 2. ESN — BULLETPROOF VERSION
# ===================================
class ESN:
    def __init__(self, n_reservoir=400, spectral_radius=0.95,
                 leaking_rate=0.3, input_scaling=1.0,
                 sparsity=0.05, noise=0.001, ridge=1e-7):
        
        self.n_res = n_reservoir
        self.sr = spectral_radius
        self.alpha = leaking_rate
        self.scale = input_scaling
        self.sparsity = sparsity
        self.noise = noise
        self.ridge = ridge

        # Input weight: (n_res, 1)
        self.Win = np.random.uniform(-1, 1, (n_reservoir, 1)) * input_scaling

        # Reservoir weights: sparse, scaled spectral radius
        W = np.random.uniform(-1, 1, (n_reservoir, n_reservoir))
        mask = np.random.rand(n_reservoir, n_reservoir) < sparsity
        W *= mask
        rho = np.max(np.abs(np.linalg.eigvals(W)))
        self.Wres = W * (spectral_radius / rho) if rho > 0 else W

        self.Wout = None

    def _update(self, state, u):
        u = np.array([u])  # shape (1,)
        noise = self.noise * (np.random.rand(self.n_res) - 0.5)
        pre = self.Wres @ state + self.Win @ u + noise.reshape(-1, 1)
        return (1 - self.alpha) * state + self.alpha * np.tanh(pre)

    def train(self, inputs, targets, washout=500):
        state = np.zeros((self.n_res, 1))
        X = []
        Y = []

        for t in range(len(inputs)):
            state = self._update(state, inputs[t])
            if t >= washout:
                # Build extended state: [reservoir; input; bias]
                extended = np.concatenate([state.flatten(), [inputs[t]], [1.0]])
                X.append(extended)
                Y.append(targets[t])

        X = np.array(X)        # shape (steps, n_res + 2)
        Y = np.array(Y).reshape(-1, 1)

        # Ridge regression: Wout = (X^T X + λI)^-1 X^T Y
        I = np.eye(X.shape[1])
        self.Wout = np.linalg.solve(X.T @ X + self.ridge * I, X.T @ Y).flatten()

    def predict(self, start_u, length, teacher_forcing=False, true_seq=None):
        state = np.zeros((self.n_res, 1))
        state = self._update(state, start_u)  # warm-up
        preds = []
        u = start_u

        for t in range(length):
            extended = np.array(state.flatten().tolist() + [u, 1.0])
            pred = np.dot(extended, self.Wout)
            preds.append(pred)

            if teacher_forcing and true_seq is not None:
                u = true_seq[t]
            else:
                u = pred
            state = self._update(state, u)

        return np.array(preds)

# ===================================
# 3. TRAIN & TEST
# ===================================
esn = ESN(
    n_reservoir=400,
    spectral_radius=0.95,
    leaking_rate=0.4,
    input_scaling=1.0,
    sparsity=0.05,
    noise=0.001,
    ridge=1e-7
)

print("Training ESN...")
esn.train(train_data[:-1], train_data[1:], washout=500)
print("Training complete!")

# One-step ahead (teacher forcing)
print("Testing one-step prediction...")
pred_teacher = esn.predict(
    test_data[0],
    length=len(test_data)-1,
    teacher_forcing=True,
    true_seq=test_data[1:]
)

nrmse = np.sqrt(np.mean((pred_teacher - test_data[1:])**2)) / np.std(test_data[1:])
print(f"One-step NRMSE: {nrmse:.6f}")

# Free-run
print("Running free prediction...")
pred_free = esn.predict(test_data[0], length=1000, teacher_forcing=False)
true_free = logistic_map(r, test_data[0], 1000)

error = np.abs(pred_free - true_free)
div_step = np.where(error > 0.05)[0]
steps = div_step[0] if len(div_step) > 0 else 1000
print(f"Free-run accurate (<0.05 error) for {steps} steps")

# ===================================
# 4. PLOTS
# ===================================
plt.figure(figsize=(14, 6))

plt.subplot(1, 3, 1)
plt.plot(test_data[1:500], 'b', label='True', linewidth=1.5)
plt.plot(pred_teacher[:499], 'r--', label='ESN (1-step)', linewidth=1.5)
plt.title(f'One-step Prediction\nNRMSE = {nrmse:.5f}')
plt.legend(); plt.grid(True)

plt.subplot(1, 3, 2)
plt.plot(true_free[:500], 'b', label='True', linewidth=1.5)
plt.plot(pred_free[:500], 'r--', label='ESN Free Run', linewidth=1.5)
plt.title(f'Free Run Prediction\nAccurate for {steps} steps')
plt.legend(); plt.grid(True)

plt.subplot(1, 3, 3)
plt.plot(true_free[:2000], true_free[1:2001], 'b.', ms=1, alpha=0.6, label='True')
plt.plot(pred_free[:2000], pred_free[1:2001], 'r.', ms=1, alpha=0.6, label='ESN')
x = np.linspace(0, 1, 200)
plt.plot(x, r*x*(1-x), 'k-', lw=1.5, label='y=4x(1-x)')
plt.xlabel('x(n)'); plt.ylabel('x(n+1)')
plt.title('Attractor Reconstruction')
plt.legend(); plt.grid(True)

plt.tight_layout()
plt.show()

Training ESN...


In [ ]:
for scaling in [0.1, 0.5, 1.0, 2.0, 5.0]:
    esn = ESN(input_scaling=scaling, n_reservoir=500, spectral_radius=0.97)
    esn.train(train_data[:-1], train_data[1:])
    predree, _ = esn.predict(test_data[:1000].copy(), teacher_forcing=False)
    error_steps = np.where(np.abs(predree - logistic_map(4.0, test_data[0], 1000)) > 0.05)[0]
    steps = error_steps[0] if len(error_steps)>0 else 1000
    print(f"input_scaling={scaling:4.1f} → accurate for {steps} steps")